In [1]:
import pickle
import yaml
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook, tqdm

from skopt import forest_minimize
from skopt.plots import plot_convergence
from skopt.utils import use_named_args

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

import xgboost
from xgboost import XGBClassifier

import warnings
from multiprocessing import cpu_count

from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer
from skopt import gp_minimize,gbrt_minimize
from src.functions import (
    generate_stratified_folds,
    cross_validate_score,
    generate_space_dimension,
    update_parameter_yaml
)

from xtlearn.utils import load_pickle, dump_pickle
from copy import deepcopy

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

In [2]:
n_jobs = cpu_count()
inp_path = "data/"
sub_path = "submissions/"
%run src/import_parameters.py
TARGET = pd.read_csv(sub_path + "submission_41_teste.csv")["predicted"]

### Reading data from Porto Seguro Data Challenge 2021
* Remover id

In [3]:
metadata = pd.read_csv(inp_path + 'metadata.csv')
test = pd.read_csv(inp_path + 'test.csv')
train = pd.read_csv(inp_path + 'train.csv')
train = train.sample(frac=1, random_state=0).reset_index(drop=True)  # shuffling dataset
sub = pd.read_csv(inp_path + 'submission_sample.csv')
feature_selection = pd.read_csv("data/f1_score_for_features.csv")

###########################

# Excluindo colunas por terem muitos missings
train = train.drop(columns = ["var60","var65","var66"],errors="ignore")
test = test.drop(columns = ["var60","var65","var66"],errors="ignore")


cols_to_drop = feature_selection[feature_selection["action"]=="drop"]["feature"].to_list()
train = train.drop(columns = cols_to_drop,errors="ignore")
test = test.drop(columns = cols_to_drop,errors="ignore")

# train = train.replace(-999,np.nan)
# nans_count = train.isna().mean().to_dict()

# cols_to_flag = []

# for k in dict(filter(lambda x: x[1] > 0.10,nans_count.items())):
#     if nans_count[k] > 0:
#         #cols_to_flag.append(k)
#         train[k+"_nan_flag"] = train[k].isna().astype(int)
#         test[k+"_nan_flag"] = test[k].isna().astype(int)

## Função de Treino

In [23]:
def score_function(cv, corr):
#     std_cv = 0.012
#     mean_cv = -0.71
#     std_corr = 0.06
#     mean_corr = 0.99
    std_cv = 0.06
    mean_cv = -0.686
    std_corr = 0.15
    mean_corr = 0.995

    score_cv = cv  * np.exp(-0.5 * (cv - mean_cv) ** 2 / std_cv ** 2)
    score_corr = corr * np.exp(-0.5 * (corr - mean_corr) ** 2 / std_corr ** 2)
    
    #print("cv =",cv,"  |  corr =",corr)

    return np.sqrt(0.3 * (score_cv / mean_cv - 1) ** 2 + 0.7 * (score_corr / mean_corr - 1) ** 2)

In [24]:
def predict(model, X_test, threshold=0.5):
    predict_prob = model.predict_proba(X_test)[:, 1]
    condition = lambda x: x > threshold
    vec_condition = np.vectorize(condition)
    return np.where(vec_condition(predict_prob), 1.0, 0.0)

In [25]:
def cross_validate_score(
    X,
    y,
    estimator,
    n_folds=5,
    scoring=f1_score,
    threshold=0.3,
    random_state=42,
    verbose=0,
    fit_params={},
):

    scores = []

    temp = X.assign(y=y)

    temp["fold"] = generate_stratified_folds(temp, n_folds=n_folds)

    iterator = (
        range(n_folds) if verbose < 1 else tqdm(range(n_folds), desc="Cross validation")
    )

    for fold in iterator:

        # Separando os dados de treinamento para essa fold
        train_data = temp[temp["fold"] != fold].copy()

        # Separando os dados de teste para esse fold
        test_data = temp[temp["fold"] == fold].copy()

        cv = train_scoring(
            estimator, train_data, test_data, scoring, threshold=threshold, fit_params=fit_params
        )

        scores.append(cv)

    cv = np.mean(scores)

    return -cv

In [26]:
def train_scoring(estimator,train_data,test_data, scoring, threshold=0.5,fit_params = {}):
        
        opt_drop = dict(columns=["fold", "y","id"],errors='ignore')
        
        X_train = train_data.drop(**opt_drop).values

        X_test = test_data.drop(**opt_drop).values

        y_train = train_data["y"].values

        y_test = test_data["y"].values
        
        fit_params["eval_set"] = [(X_test, y_test)]

        estimator.fit(X_train, y_train, **fit_params)
        
        prob_test = estimator.predict_proba(X_test)[:, -1]
        
        y_pred = prob_test > threshold
        
        cv = scoring(y_test, y_pred)

        return cv

    

## Função de Escoragem

In [27]:
def S(par, limits):

    par_scaled = deepcopy(par)

    params = {
        k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
        for i, k in enumerate(par_scaled)
    }

    threshold = THRESHOLD
    if "threshold" in params:
        threshold = params.pop("threshold")

    estimator = XGBClassifier(
        n_jobs=n_jobs, random_state=RANDOM_STATE, **params, **DISC_PAR
    )

    fit_params = {
        "early_stopping_rounds": EARLY_STOPPING_ROUNDS,
        "eval_metric": EVAL_METRIC,
        "verbose": VERBOSE,
    }

    cv = cross_validate_score(
        X=train.drop(["y", "id"], 1,errors="ignore"),
        y=train["y"],
        estimator=estimator,
        fit_params=fit_params,
        n_folds=N_FOLDS,
        scoring=SCORING,
        threshold=threshold,
        random_state=RANDOM_STATE,
        verbose=0,
    )

    # Separando os dados de treinamento para essa fold
    train_data = train.copy()

    # Separando os dados de teste para esse fold
    test_data = test.copy()

    X_train = train_data.drop(columns=["fold", "y", "id"], errors="ignore").values

    X_test = test_data.drop(columns=["fold", "y", "id"], errors="ignore").values

    y_train = train_data["y"].values

    y_test = TARGET.values

    fit_params.pop("eval_set")
    fit_params["eval_set"] = [(X_test, y_test)]

    estimator.fit(X_train, y_train, **fit_params)

    y_pred = predict(estimator, X_test, threshold=threshold).astype(int)

    corr = pd.DataFrame({0: TARGET, 1: y_pred}).corr()[0][1]

    if pd.isnull(corr):
        corr = 0

    score = score_function(cv, corr)
    # score = 0.5 * (cv / 0.71 + 1) ** 2 + 0.5 * (corr / 0.99 - 1) ** 2

    return score

## Otimizadores

In [28]:
def local_opt(par, n_iter=20, epsilon=0.1, alpha=0.1, alpha_end=None, n_tries=4):

    par.update({"threshold": THRESHOLD})
    limits.update({"threshold": [0.1, 0.9]})

    par_scaled = {
        k: (par[k] - limits[k][0]) / (limits[k][1] - limits[k][0])
        for i, k in enumerate(par)
    }

    eval_ = []

    S0 = S(par_scaled, limits)

    print("Input Score:", S0)

    learn_rate = deepcopy(alpha)

    cont = 0
    results = [[0,par,S0]]
    for j in range(n_iter):

        print("\n\n###############################################")
        print("Iteração:", j," | Score atual:",S0)

        # Gradiente
        grad = np.zeros(len(par_scaled))
        for dim, k in enumerate(par):
            new_par = {
                k: np.clip(par_scaled[k] + int(i == dim) * epsilon, 0, 1)
                for i, k in enumerate(par_scaled)
            }
            new_S = S(new_par, limits)
            grad[dim] = (new_S - S0) / epsilon

        grad = grad / np.linalg.norm(grad)


        # Atualizando learning_rate
        if alpha_end is not None:
            learn_rate = alpha * (alpha_end / alpha) ** (j / n_iter)

        old_S = S0
        tries_results = [(par_scaled,S0)]
        
        for ind in range(n_tries):

            # Se n_tries for 2, então, primeiramente o fator será 1 e depois 0.5
            # Se n_tries for 3, então, primeiramente o fator será 1 0.66 e 0.33
            
            factor = 1 - ind/n_tries

            new_par_scaled = {
                k: np.clip(par_scaled[k] - factor * learn_rate * grad[i], 0, 1)
                for i, k in enumerate(par_scaled)
            }

            new_S = S(new_par_scaled, limits)
            tries_results.append((new_par_scaled,new_S))

            print("\t\t>>> New S:", new_S)

            if new_S <= S0:
                break
                
            if new_S == old_S:
                break
            
            old_S = new_S
            
        indmin = np.argmin([tup[1] for tup in tries_results])
        new_par_scaled,new_S = tries_results[indmin]

        if new_S < S0:
            S0 = new_S
            par_scaled = new_par_scaled
        else:
            print("Random!")
            prop_par_scaled = {
                k: np.clip(
                    par_scaled[k]
                    - np.random.uniform(
                        -factor * learn_rate * grad[i], factor * learn_rate * grad[i]
                    ),
                    0,
                    1,
                )
                for i, k in enumerate(par_scaled)
            }
            prop_S = S(prop_par_scaled, limits)
            P0 = np.exp(S0)
            new_P = np.exp(prop_S)
            A = min(1, new_P / P0)
            u = np.random.random()
            if u <= A:
                if prop_S < new_S:
                    S0 = prop_S
                    par_scaled = prop_par_scaled
                else:
                    S0 = new_S
                    par_scaled = new_par_scaled
                    
                
        

        par = {
            k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
            for i, k in enumerate(par_scaled)
        }
    
        results.append([j+1,par,S0])
        print("iter =", j, "S0 =", S0, " | alpha =", learn_rate)
        print("parameters:",par)

        eval_.append([j, par_scaled, S0])

    eval_.append([n_iter, par_scaled, S(par_scaled, limits)])

    ind = (
        pd.DataFrame({0: np.array(eval_)[:, 0], 1: np.array(eval_)[:, 2]})
        .sort_values(1)
        .iloc[0][0]
    )

    par_scaled = eval_[ind][1]

    params = {
        k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
        for i, k in enumerate(par_scaled)
    }
    
    return params, eval_[ind][2], results

In [29]:
def opt(par,n_iter=20,epsilon = 0.1,alpha = 0.1):

    par.update({"threshold": THRESHOLD})
    limits.update({"threshold": [0.1, 0.9]})

    par_scaled = {
        k: (par[k] - limits[k][0]) / (limits[k][1] - limits[k][0])
        for i, k in enumerate(par)
    }

    eval_ = []

    S0 = S(par_scaled, limits)

    grad = np.zeros(len(par_scaled))

    learn_rate = deepcopy(alpha)

    for j in range(n_iter):

        new_par_scaled = {
            k: np.clip(par_scaled[k] - learn_rate * grad[i], 0, 1)
            for i, k in enumerate(par_scaled)
        }

        new_S = S(new_par_scaled, limits)

        # apagar
        S0 = new_S
        par_scaled = new_par_scaled

        if new_S < S0:
            S0 = new_S
            par_scaled = new_par_scaled
        else:
            new_par_scaled = {
                k: np.clip(par_scaled[k] - np.random.uniform(-learn_rate,learn_rate), 0, 1)
                for i, k in enumerate(par_scaled)
            }
            new_S = S(new_par_scaled, limits)
            P0 = np.exp(S0)
            new_P = np.exp(new_S)
            A = min(1,new_P / P0)
            u = np.random.random()
            if u <= A:
                S0 = new_S
                par_scaled = new_par_scaled

        print(S0)

        grad = np.zeros(len(par_scaled))
        eval_.append([j, par_scaled, S0])
        for dim, k in enumerate(par):
            new_par = {
                k: np.clip(par_scaled[k] + int(i == dim) * epsilon, 0, 1)
                for i, k in enumerate(par_scaled)
            }
            new_S = S(new_par, limits)
            grad[dim] = (new_S - S0) / epsilon

        grad = grad / np.linalg.norm(grad)


    eval_.append([n_iter, par_scaled, S(par_scaled, limits)])
    
    ind = (
        pd.DataFrame({0: np.array(eval_)[:, 0], 1: np.array(eval_)[:, 2]})
        .sort_values(1)
        .iloc[0][0]
    )

    par_scaled = eval_[ind][1]
    
    params = {
        k: (par_scaled[k] * (limits[k][1] - limits[k][0]) + limits[k][0])
        for i, k in enumerate(par_scaled)
    }
    
    return params, eval_[ind][2]



## Importando Parâmetros

In [30]:
from xtlearn.utils import load_pickle, dump_pickle

%run src/import_parameters.py

# Parameters space
parameters = yaml.safe_load(open("data/parameters.yaml", "r"))
space = [generate_space_dimension(x) for x in parameters]
PARAMETER_NAMES = [elem.name for elem in space]

params_real = list(filter(lambda x: x['type']=='real',parameters))
params_disc = list(filter(lambda x: x['type']!='real',parameters))

params = {x['parameter']:x['estimate']for x in params_real}
limits = {x['parameter']:x['range']for x in params_real}
steps = {x['parameter']:x['step']for x in params_real}

DISC_PAR = {
    'n_estimators': 100, 
    'max_depth': 6, 
    'num_parallel_tree': 2,
    'max_delta_step': 0
}
space = [generate_space_dimension(x) for x in params_real]
PARAMETER_NAMES = [elem.name for elem in space]

# x0 = list(params.values())
# y0 = 2

# dump_pickle(space,"data/best_space.pkl")
# dump_pickle(PARAMETER_NAMES,"data/best_PARAMETER_NAMES.pkl")
# dump_pickle(x0,"data/best_x0.pkl")
# dump_pickle(score,"data/best_y0.pkl")

## Otimização Local

In [38]:
%%time

PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES, x0))

list_mcmc = [[params,score]]
for i in range(15):
    params_temp, score_temp, result_temp = local_opt(
        params,
        n_iter=10,
        epsilon=0.1 * 10 ** np.random.uniform(-4, 0),
        alpha=0.1 * 10 ** np.random.uniform(-4, 0),
        alpha_end=0.1 * 10 ** np.random.uniform(-4, 0),
        n_tries=50,
    )

    indmin = np.argmin([tup[2] for tup in result_temp])
    *_, params_temp, score_temp = result_temp[indmin]

    if score > score_temp:
        score = score_temp
        params = params_temp
        result = result_temp
        list_mcmc.append([params, score])

    else:
        A = np.exp(-score_temp) / np.exp(-score)

        u = np.random.random()

        if A > u:
            score = score_temp
            params = params_temp
            result = result_temp
            list_mcmc.append([params, score])

indmin = np.argmin([tup[1] for tup in list_mcmc])
params, score = list_mcmc[indmin]

x0 = [x[1] for x in filter(lambda x: x[0] != "threshold",params.items())]
dump_pickle(x0,"data/best_x0.pkl")
dump_pickle(score,"data/best_y0.pkl")
THRESHOLD = params["threshold"]
dump_pickle(THRESHOLD,"data/best_threshold.pkl")

Input Score: 0.005000500050392699


###############################################
Iteração: 0  | Score atual: 0.005000500050392699
		>>> New S: 0.008136600104362148
		>>> New S: 0.008354610247143643
		>>> New S: 0.009212453059255157
		>>> New S: 0.006672279349611014
		>>> New S: 0.008277360460341495
		>>> New S: 0.01090125388950449
		>>> New S: 0.009360890612159016
		>>> New S: 0.008647543180503228
		>>> New S: 0.007509650049200392
		>>> New S: 0.009221138335235739
		>>> New S: 0.007876221548612023
		>>> New S: 0.009657930800602467
		>>> New S: 0.007853118719098304
		>>> New S: 0.008586929346846463
		>>> New S: 0.00778450473471031
		>>> New S: 0.009227080128133457
		>>> New S: 0.006991822638130706
		>>> New S: 0.008155667606832374
		>>> New S: 0.007505031190852203
		>>> New S: 0.00896606744065109
		>>> New S: 0.00896606744065109
Random!
iter = 0 S0 = 0.005000500050392699  | alpha = 0.0013293906288690772
parameters: {'learning_rate': 0.053318495963581805, 'min_child_weight': 0.3608109

		>>> New S: 0.008644644065707498
		>>> New S: 0.009213119837240121
		>>> New S: 0.009526121844910803
		>>> New S: 0.007950891140781432
		>>> New S: 0.007410764392247274
		>>> New S: 0.009259582278473585
		>>> New S: 0.00824567360890201
		>>> New S: 0.007748321831398522
		>>> New S: 0.009582632750691576
		>>> New S: 0.007511468908969972
		>>> New S: 0.008166335023960412
		>>> New S: 0.007579389223686703
		>>> New S: 0.009907408734873438
		>>> New S: 0.006531561699236547
Random!
iter = 4 S0 = 0.00431218176085127  | alpha = 0.003793748188161333
parameters: {'learning_rate': 0.053316503758725875, 'min_child_weight': 0.36078346379514525, 'subsample': 0.9358551308029192, 'reg_alpha': 0.9370776843748331, 'reg_lambda': 0.967336118342639, 'colsample_bynode': 0.8332353261601001, 'colsample_bytree': 0.33460185330155934, 'threshold': 0.30426698430394683}


###############################################
Iteração: 5  | Score atual: 0.00431218176085127
		>>> New S: 0.010447053991726568
		>>> New S:

		>>> New S: 0.009219802606843883
		>>> New S: 0.012435184987646437
		>>> New S: 0.012265660744791946
		>>> New S: 0.011632511932756842
		>>> New S: 0.011275854173026002
		>>> New S: 0.010707719089858755
		>>> New S: 0.008571973015788104
		>>> New S: 0.010938325867834935
		>>> New S: 0.009830385491514132
		>>> New S: 0.011585981429683635
		>>> New S: 0.008877919155490533
		>>> New S: 0.010139624363865857
		>>> New S: 0.011604939069162318
		>>> New S: 0.009960958722683886
		>>> New S: 0.009473650480940655
		>>> New S: 0.010375265134350444
		>>> New S: 0.009303659957266919
		>>> New S: 0.00689671432425265
		>>> New S: 0.007606236539182732
		>>> New S: 0.010624933316740243
		>>> New S: 0.009589159036753142
		>>> New S: 0.009143494027534223
		>>> New S: 0.008862783536351015
		>>> New S: 0.007365635725883402
		>>> New S: 0.007988851069990465
		>>> New S: 0.0058412033945246244
		>>> New S: 0.008618930041187453
		>>> New S: 0.008060831934755276
Random!
iter = 8 S0 = 0.00431218176085127  | alp

		>>> New S: 0.022546860400412972
		>>> New S: 0.022145649131655905
		>>> New S: 0.01833981466991784
		>>> New S: 0.018565530638479977
		>>> New S: 0.01660459368664595
		>>> New S: 0.015552993415033725
		>>> New S: 0.01654875426643435
		>>> New S: 0.016390742993260723
		>>> New S: 0.01699694120730222
		>>> New S: 0.014943541109681543
		>>> New S: 0.016568642628595916
		>>> New S: 0.013306458708405232
		>>> New S: 0.015534451592623441
		>>> New S: 0.015675966145093576
		>>> New S: 0.014246573903269657
		>>> New S: 0.01677682656635682
		>>> New S: 0.015660744346545154
		>>> New S: 0.017166059603827014
		>>> New S: 0.014312850071464994
		>>> New S: 0.012661867903033756
		>>> New S: 0.01438243633193839
		>>> New S: 0.01289521480615084
		>>> New S: 0.012509168245178821
		>>> New S: 0.01217960016771611
		>>> New S: 0.014397201896799673
		>>> New S: 0.013508235766325595
		>>> New S: 0.012807850044337892
		>>> New S: 0.011888520835718163
		>>> New S: 0.014822453777343295
		>>> New S: 0.0135901

		>>> New S: 0.0075759786972283485
		>>> New S: 0.0051533469321252265
		>>> New S: 0.007693003424357052
		>>> New S: 0.00765204711551059
		>>> New S: 0.005915674659572015
		>>> New S: 0.0075104640987836676
		>>> New S: 0.009275192116141254
		>>> New S: 0.00755303696198012
		>>> New S: 0.006609410653281672
		>>> New S: 0.008374044540855361
		>>> New S: 0.009116343759681997
		>>> New S: 0.007875465970431527
		>>> New S: 0.009422369382423515
		>>> New S: 0.00852309125554949
		>>> New S: 0.009148708341881421
		>>> New S: 0.007984766530100546
		>>> New S: 0.007946548507778237
		>>> New S: 0.008331598293412883
		>>> New S: 0.008332766688562043
		>>> New S: 0.007190018054204287
		>>> New S: 0.009129454561199535
		>>> New S: 0.009875048688156855
		>>> New S: 0.008260952067504341
		>>> New S: 0.008660255636966607
		>>> New S: 0.007285131325797729
		>>> New S: 0.008172583701137376
		>>> New S: 0.007692145965937157
		>>> New S: 0.009017702423392856
		>>> New S: 0.0085973221057862
		>>> New S: 0.0

...

		>>> New S: 0.007695647038368183
		>>> New S: 0.008752615833740318
		>>> New S: 0.008008282610627084
		>>> New S: 0.007538332984377334
		>>> New S: 0.009457582364374445
Random!
iter = 3 S0 = 0.0032283499579481823  | alpha = 0.002149398551599544
parameters: {'learning_rate': 0.053322618456427134, 'min_child_weight': 0.3608401158059781, 'subsample': 0.9358430939491236, 'reg_alpha': 0.9371231490846818, 'reg_lambda': 0.9672607666228303, 'colsample_bynode': 0.8332353261601001, 'colsample_bytree': 0.33474074180245333, 'threshold': 0.30436026216615597}


###############################################
Iteração: 4  | Score atual: 0.0032283499579481823
		>>> New S: 0.010517380583497231
		>>> New S: 0.008384401460978342
		>>> New S: 0.01087038829194232
		>>> New S: 0.009804685515129741
		>>> New S: 0.012685863475909259
		>>> New S: 0.011677686073397751
		>>> New S: 0.009693604948527865
		>>> New S: 0.010510711167173708
		>>> New S: 0.013335657452985036
		>>> New S: 0.010340489122492153
		>>> Ne

		>>> New S: 0.01046506464427867
		>>> New S: 0.01107027543900047
		>>> New S: 0.00768684867865001
		>>> New S: 0.012109883767254123
		>>> New S: 0.010665708657597833
		>>> New S: 0.009537534064816881
		>>> New S: 0.01137695860693762
		>>> New S: 0.010374384817009428
		>>> New S: 0.010543442432171058
		>>> New S: 0.009198743768821668
		>>> New S: 0.008749299640461084
		>>> New S: 0.006812991069917192
		>>> New S: 0.007578231589040512
		>>> New S: 0.00803133150338765
		>>> New S: 0.008872781850554215
		>>> New S: 0.010345434200583144
		>>> New S: 0.009288173575552726
		>>> New S: 0.0071001930713544435
		>>> New S: 0.009002999513592562
		>>> New S: 0.008167912075099025
Random!
iter = 7 S0 = 0.0032283499579481823  | alpha = 0.005944072960756381
parameters: {'learning_rate': 0.053322618456427134, 'min_child_weight': 0.3608401158059781, 'subsample': 0.9358430939491236, 'reg_alpha': 0.9371231490846818, 'reg_lambda': 0.9672607666228303, 'colsample_bynode': 0.8332353261601001, 'colsample_bytre

		>>> New S: 0.09553560100773818
		>>> New S: 0.09728604200020421
		>>> New S: 0.09433107084300958
		>>> New S: 0.08812427598991679
		>>> New S: 0.08475292549769706
		>>> New S: 0.08591060908613465
		>>> New S: 0.07584967303494426
		>>> New S: 0.07161947260060282
		>>> New S: 0.07396346933017638
		>>> New S: 0.06768239397456999
		>>> New S: 0.06801519325659768
		>>> New S: 0.06310464571938806
		>>> New S: 0.05776072855535105
		>>> New S: 0.05572925690649072
		>>> New S: 0.05359920706843599
		>>> New S: 0.05311607547973563
		>>> New S: 0.0473998537960536
		>>> New S: 0.04593066297228067
		>>> New S: 0.048139616744587284
		>>> New S: 0.043221086358065326
		>>> New S: 0.027599409899357396
		>>> New S: 0.022988662330773024
		>>> New S: 0.01997658170910103
		>>> New S: 0.02163136279040216
		>>> New S: 0.019288631487973218
		>>> New S: 0.018605434942566505
		>>> New S: 0.017041936081835893
		>>> New S: 0.013752261600100974
		>>> New S: 0.013301201723602392
		>>> New S: 0.013763848411351701
	

		>>> New S: 0.11689126523837015
		>>> New S: 0.11192779338981096
		>>> New S: 0.10840542105115303
		>>> New S: 0.10515167477126801
		>>> New S: 0.10127648760956369
		>>> New S: 0.09834990453126533
		>>> New S: 0.10103737493401443
		>>> New S: 0.09556462333832928
		>>> New S: 0.09279580544510145
		>>> New S: 0.08651069787411246
		>>> New S: 0.08168558091589326
		>>> New S: 0.08513199159292217
		>>> New S: 0.07752436256872249
		>>> New S: 0.07294527183921984
		>>> New S: 0.07032540874828638
		>>> New S: 0.07063213391033966
		>>> New S: 0.06652595300105464
		>>> New S: 0.06512005507951293
		>>> New S: 0.0605966537847526
		>>> New S: 0.06171925327340097
		>>> New S: 0.05440267975439194
		>>> New S: 0.056596743563356734
		>>> New S: 0.049953939377228085
		>>> New S: 0.05317829597026038
		>>> New S: 0.052060068112541116
		>>> New S: 0.047180619570922816
		>>> New S: 0.04457254625906969
		>>> New S: 0.042290182942774125
		>>> New S: 0.027311861716442856
		>>> New S: 0.023827840384436305
		>>

KeyboardInterrupt: 

In [46]:
params = {
    "learning_rate": 0.053322618456427134,
    "min_child_weight": 0.3608401158059781,
    "subsample": 0.9358430939491236,
    "reg_alpha": 0.9371231490846818,
    "reg_lambda": 0.9672607666228303,
    "colsample_bynode": 0.8332353261601001,
    "colsample_bytree": 0.33474074180245333,
    "threshold": 0.30436026216615597,
}

score = 0.0032283499579481823

x0 = [x[1] for x in filter(lambda x: x[0] != "threshold", params.items())]
THRESHOLD = params["threshold"]
dump_pickle(x0, "data/best_x0.pkl")
dump_pickle(score, "data/best_y0.pkl")
dump_pickle(THRESHOLD, "data/best_threshold.pkl")

True

In [47]:
score

0.0032283499579481823

In [36]:
# PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
# THRESHOLD = load_pickle("data/best_threshold.pkl")
# x0 = load_pickle("data/best_x0.pkl")
# score = load_pickle("data/best_y0.pkl")
# params = dict(zip(PARAMETER_NAMES,x0))


# f(params, limits)

# MCMC

In [48]:
load_pickle("data/best_y0.pkl")

0.0032283499579481823

In [ ]:
%%time
from tqdm import tqdm

PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES, x0))


def f(params, limits, index):
    par_scaled = {
        k: (params[k] - limits[k][0]) / (limits[k][1] - limits[k][0])
        for i, k in enumerate(params)
    }
    s = S(par_scaled, limits)
    result = np.exp(-s)

    print("\n#################################################")
    print(f"{index} - score = {s}  |  prob = {result}")
    print(params)

    return result


params = dict(zip(PARAMETER_NAMES, x0))
P = f(params, limits, 0)

params_max = params
P_max = P

list_mcmc = [(0, params, P, P_max)]
for ind in range(550):

    new_params = {
        k: np.clip(
            params[k] + 0.01 * np.random.randn() * steps[k], limits[k][0], limits[k][1]
        )
        for k in params
    }

    new_P = f(new_params, limits, ind + 1)

    A = min(1, new_P / P)
    u = np.random.random()

    if u < A:
        P = new_P
        params = new_params

    if P > P_max:
        P_max = P
        params_max = params

    list_mcmc.append((ind, params, P, P_max))


x0 = [x[1] for x in filter(lambda x: x[0] != "threshold", params_max.items())]
# THRESHOLD = params_max["threshold"]
dump_pickle(x0, "data/best_x0.pkl")
dump_pickle(score, "data/best_y0.pkl")
dump_pickle(THRESHOLD, "data/best_threshold.pkl")

## Otimização Skopt

In [ ]:
@use_named_args(space)
def train_function_test(**params):

    print(f"Testing parameters: {params.values()}")

    try:
        estimator = XGBClassifier(
            n_jobs=n_jobs,
            random_state=RANDOM_STATE,
            **params,
            **DISC_PAR,
        )

        fit_params = {
            "early_stopping_rounds": EARLY_STOPPING_ROUNDS,
            "eval_metric": EVAL_METRIC,
            "verbose": VERBOSE,
        }

        cv = cross_validate_score(
            X=train.drop(["y","id"], 1,errors="ignore"),
            y=train["y"],
            estimator=estimator,
            fit_params=fit_params,
            n_folds=N_FOLDS,
            scoring=SCORING,
            threshold=THRESHOLD,
            random_state=RANDOM_STATE,
            verbose=0,
        )
        
        
        fit_params["eval_set"] = [(test.drop("id",1,errors="ignore"), TARGET)]

        estimator.fit(train.drop(["y","id"], 1,errors="ignore"), train["y"])

        y_pred = predict(estimator, test.drop("id",1,errors="ignore"), threshold=THRESHOLD).astype(int)

        corr = pd.DataFrame({0: TARGET, 1: y_pred}).corr()[0][1]
        
        print(cv,corr)

        # corr = 1 - np.abs(TARGET-y_pred).mean()

        # score = 0.5 * (cv / 0.71 + 1) ** 2 + 0.5 * (corr / 0.99 - 1) ** 2
        score = score_function(cv, corr)

        if pd.isnull(score):
            score = 1
        else:

            pd.DataFrame([0] + list(params.values()) + [score]).T.to_csv(
                "data/optimization.csv", index=False, mode="a", header=False
            )

        print(f"\n\t<<<<< cv = {cv}   |   corr = {corr} |  score = {score}>>>>>")
        print(f"\n\t{params}\n")

        return score

    except Exception as err:
        print(err)
        return 1

In [ ]:
%%time
N_CALLS = 1200
N_RANDOM_STARTS = 100

# results = pd.read_csv("data/optimization.csv").sort_values('score')
# x0 = list(results.drop(["iteration", "score"], 1).iloc[0].values)
# y0 = results["score"].iloc[0].

space = load_pickle("data/best_space.pkl")
PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")


res_gp = gbrt_minimize(
# res_gp = gp_minimize(
    #train_function,
    train_function_test,
    space,
    x0=x0,
    y0=score,
    random_state=None,
    verbose=1,
    n_calls=N_CALLS,
    n_random_starts=N_RANDOM_STARTS,
    n_jobs=n_jobs,
)

x0 = res_gp.x
score = res_gp.fun

dump_pickle(x0,"data/best_x0.pkl")
dump_pickle(score,"data/best_y0.pkl")

# xgb_best_params = res_gp.x

# print(f"Best found params: {xgb_best_params}")

# plot_convergence(res_gp)

# results = (
#     pd.DataFrame(res_gp.x_iters, columns=PARAMETER_NAMES)
#     .assign(score=res_gp.func_vals)
#     .reset_index()
#     .rename(columns={"index": "iteration"})
#     .sort_values("score")
# )

# results.to_csv("data/optimization.csv", index=False, mode='a',header=False)

#update_parameter_yaml(results,parameters)

# display(results)

# Testes

In [49]:
%%time


PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES, x0))


par = deepcopy(params)

# threshold = 0.3
threshold = THRESHOLD


if "threshold" in par:
    threshold = par.pop("threshold")

estimator = XGBClassifier(
    n_jobs=n_jobs,
    random_state=RANDOM_STATE,
    **par,
    **DISC_PAR
)

print(estimator)

fit_params = {
    "early_stopping_rounds": EARLY_STOPPING_ROUNDS,
    "eval_metric": EVAL_METRIC,
    "verbose": VERBOSE,
}

cv = cross_validate_score(
    X=train.drop(["y","id"], 1,errors="ignore"),
    y=train["y"],
    estimator=estimator,
    fit_params=fit_params,
    n_folds=N_FOLDS,
    scoring=SCORING,
    threshold=threshold,
    random_state=RANDOM_STATE,
    verbose=0,
)
print(cv)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=0.8332353261601001,
              colsample_bytree=0.33474074180245333, gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.053322618456427134, max_delta_step=0, max_depth=6,
              min_child_weight=0.3608401158059781, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=16,
              num_parallel_tree=2, random_state=42,
              reg_alpha=0.9371231490846818, reg_lambda=0.9672607666228303,
              scale_pos_weight=None, subsample=0.9358430939491236,
              tree_method=None, validate_parameters=None, verbosity=None)
-0.6828798328977518
CPU times: user 30.1 s, sys: 80.1 ms, total: 30.2 s
Wall time: 2.08 s


In [50]:
# Separando os dados de treinamento para essa fold
train_data = train.copy()

# Separando os dados de teste para esse fold
test_data = test.copy()

X_train = train_data.drop(columns=["fold", "y","id"],errors='ignore').values

X_test = test_data.drop(columns=["fold", "y","id"],errors='ignore').values

y_train = train_data["y"].values

y_test = TARGET.values

fit_params.pop('eval_set')
fit_params["eval_set"] = [(X_test, y_test)]

estimator.fit(X_train, y_train, **fit_params)

y_pred = predict(estimator, X_test, threshold=threshold).astype(int)

corr = pd.DataFrame({0: TARGET, 1: y_pred}).corr()[0][1]

if pd.isnull(corr):
    corr = 0

print(corr)

score = score_function(cv,corr)
    
#score = 0.5 * (cv / 0.71 + 1) ** 2 + 0.5 * (corr / 0.99 - 1) ** 2

0.9950592420085194


In [51]:
pd.read_csv(sub_path + "submission_41_teste.csv")[["id"]].assign(predicted=y_pred).to_csv(
    sub_path + "submission_57_teste.csv", index=False
)

In [52]:
np.corrcoef(
pd.read_csv(sub_path + "submission_41_teste.csv")['predicted'],
pd.read_csv(sub_path + "submission_57_teste.csv")['predicted'])

array([[1.        , 0.99505924],
       [0.99505924, 1.        ]])

# Testes

In [23]:
%%time


PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES, x0))


par = deepcopy(params)

# threshold = 0.3
threshold = THRESHOLD


if "threshold" in par:
    threshold = par.pop("threshold")

estimator = XGBClassifier(
    n_jobs=n_jobs,
    random_state=RANDOM_STATE,
    **par,
    **DISC_PAR
)

print(estimator)

fit_params = {
    "early_stopping_rounds": EARLY_STOPPING_ROUNDS,
    "eval_metric": EVAL_METRIC,
    "verbose": VERBOSE,
}


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=0.8332353261601001,
              colsample_bytree=0.33470182526412845, gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.05332108026493601, max_delta_step=0, max_depth=6,
              min_child_weight=0.3608176224331092, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=4,
              num_parallel_tree=2, random_state=42,
              reg_alpha=0.9368230163971738, reg_lambda=0.9673847226568928,
              scale_pos_weight=None, subsample=0.9358459733547955,
              tree_method=None, validate_parameters=None, verbosity=None)
CPU times: user 1.64 ms, sys: 7.89 ms, total: 9.54 ms
Wall time: 9.21 ms


In [24]:
train_ = train.copy()
test_ = test.copy()

In [36]:
folds = train_[["id"]].assign(fold = generate_stratified_folds(train_.drop("id",1), n_folds=5))

In [37]:
folds

,id,fold
0,7674,1
1,18470,4
2,4733,4
3,32480,4
4,344,3
...,...,...
14118,32073,2
14119,23215,1
14120,19316,2
14121,23188,0


In [30]:
fold = 0

train_fold = train_[folds != fold]
test_fold = train_[folds == fold]

X_train = train_fold.drop(["id"])

In [ ]:
estimator.fit(X_train)

In [43]:
%%time

PARAMETER_NAMES = load_pickle("data/best_PARAMETER_NAMES.pkl")
THRESHOLD = load_pickle("data/best_threshold.pkl")
x0 = load_pickle("data/best_x0.pkl")
score = load_pickle("data/best_y0.pkl")
params = dict(zip(PARAMETER_NAMES, x0))

list_mcmc = [[params,score]]
for i in range(15):
    params_temp, score_temp, result_temp = local_opt(
        params,
        n_iter=10,
        epsilon=0.1 * 10 ** np.random.uniform(-4, 0),
        alpha=0.1 * 10 ** np.random.uniform(-4, 0),
        alpha_end=0.1 * 10 ** np.random.uniform(-4, 0),
        n_tries=50,
    )

    indmin = np.argmin([tup[2] for tup in result_temp])
    *_, params_temp, score_temp = result_temp[indmin]

    if score > score_temp:
        score = score_temp
        params = params_temp
        result = result_temp
        list_mcmc.append([params, score])

    else:
        A = np.exp(-score_temp) / np.exp(-score)

        u = np.random.random()

        if A > u:
            score = score_temp
            params = params_temp
            result = result_temp
            list_mcmc.append([params, score])

indmin = np.argmin([tup[1] for tup in list_mcmc])
params, score = list_mcmc[indmin]

x0 = [x[1] for x in filter(lambda x: x[0] != "threshold",params.items())]
dump_pickle(x0,"data/best_x0.pkl")
dump_pickle(score,"data/best_y0.pkl")
THRESHOLD = params["threshold"]
dump_pickle(THRESHOLD,"data/best_threshold.pkl")

Input Score: 0.005000500050392699


###############################################
Iteração: 0  | Score atual: 0.005000500050392699


KeyboardInterrupt: 